In [34]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [35]:
import netket as nk

In [36]:
print(f"NetKet version: {nk.__version__}")

NetKet version: 3.3.2.post2


In [37]:
N = 20

hi = nk.hilbert.Spin(s=1 / 2, N=N)

In [38]:
import jax
hi.random_state(jax.random.PRNGKey(0), 3)

DeviceArray([[ 1., -1., -1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1.,
              -1., -1., -1., -1.,  1.,  1., -1., -1.],
             [ 1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,
               1.,  1.,  1.,  1., -1.,  1., -1.,  1.],
             [-1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.,
               1.,  1.,  1.,  1., -1.,  1., -1.,  1.]], dtype=float32)

In [39]:
from netket.operator.spin import sigmax,sigmaz 

In [40]:
Gamma = -1
H = sum([Gamma*sigmax(hi,i) for i in range(N)])

In [41]:
V=-1
H += sum([V*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])

In [42]:
sp_h=H.to_sparse()
sp_h.shape

from scipy.sparse.linalg import eigsh

eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")

print("eigenvalues with scipy sparse:", eig_vals)

E_gs = eig_vals[0]

NameError: name 'sp_h' is not defined

In [ ]:
# Create an instance of the model. 
# Notice that this does not create the parameters.
mf_model=MF()

# Create the local sampler on the hilbert space
sampler = nk.sampler.MetropolisLocal(hi)

# Construct the variational state using the model and the sampler above.
# n_samples specifies how many samples should be used to compute expectation
# values.
vstate = nk.vqs.MCState(sampler, mf_model, n_samples=512)

In [ ]:
# you can inspect the parameters which contain the single
# variational parameter `lambda`
print(vstate.parameters)

FrozenDict({
    lambda: DeviceArray([-0.00082106], dtype=float64),
})


In [ ]:
# Expectation value: notice that it also provides an error estimate.
E = vstate.expect(H)
print(E)

-19.86 ± 0.20 [σ²=20.11, R̂=0.9937]


In [ ]:
# the energy (expectation value) is a structure with a lot of fields:
print("Mean                  :", E.mean)
print("Error                 :", E.error_of_mean)
print("Variance              :", E.variance)
print("Convergence indicator :", E.R_hat)
print("Correlation time      :", E.tau_corr)

Mean                  : -19.859484128455726
Error                 : 0.19816453949554358
Variance              : 20.105822573302195
Convergence indicator : 0.9936760107778482
Correlation time      : 0.0


In [ ]:
vstate.expect_and_grad(H)

(-19.86 ± 0.20 [σ²=20.11, R̂=0.9937],
 FrozenDict({
     lambda: DeviceArray([-0.18306634], dtype=float64),
 }))

4. Variational Monte Carlo

In [ ]:
class FFN(nn.Module):
    
    # You can define attributes at the module-level
    # with a default. This allows you to easily change
    # some hyper-parameter without redefining the whole 
    # flax module.
    alpha : int = 1
            
    @nn.compact
    def __call__(self, x):

        # here we construct the first dense layer using a
        # pre-built implementation in flax.
        # features is the number of output nodes
        # WARNING: Won't work with complex hamiltonians because
        # of a bug in flax. Use nk.nn.Dense otherwise. 
        dense = nn.Dense(features=self.alpha * x.shape[-1])
        
        # we apply the dense layer to the input
        y = dense(x)

        # the non-linearity is a simple ReLu
        y = nn.relu(y)
                
        # sum the output
        return jnp.sum(y, axis=-1)
    
model = FFN(alpha=1)

vstate = nk.vqs.MCState(sampler, model, n_samples=1008)

In [ ]:
optimizer = nk.optimizer.Sgd(learning_rate=0.1)

# Notice the use, again of Stochastic Reconfiguration, which considerably improves the optimisation
gs = nk.driver.VMC(H, optimizer, variational_state=vstate,preconditioner=nk.optimizer.SR(diag_shift=0.1))

log=nk.logging.RuntimeLog()
gs.run(n_iter=300,out=log)

ffn_energy=vstate.expect(H)
error=abs((ffn_energy.mean-eig_vals[0])/eig_vals[0])
print("Optimized energy and relative error: ",ffn_energy,error)

100%|██████████| 300/300 [00:05<00:00, 52.92it/s, Energy=-25.459 ± 0.011 [σ²=0.118, R̂=1.0007]]   


NameError: name 'eig_vals' is not defined

In [ ]:
data_jastrow = log.data
print(data_jastrow)

In [ ]:
from matplotlib import pyplot as plt

plt.errorbar(data_jastrow["Energy"].iters, data_jastrow["Energy"].Mean, yerr=data_jastrow["Energy"].Sigma)
plt.xlabel('Iterations')
plt.ylabel('Energy')

6. N-N Quantum State

In [ ]:
class FFN(nn.Module):
    
    # You can define attributes at the module-level
    # with a default. This allows you to easily change
    # some hyper-parameter without redefining the whole 
    # flax module.
    alpha : int = 1
            
    @nn.compact
    def __call__(self, x):

        # here we construct the first dense layer using a
        # pre-built implementation in flax.
        # features is the number of output nodes
        # WARNING: Won't work with complex hamiltonians because
        # of a bug in flax. Use nk.nn.Dense otherwise. 
        dense = nn.Dense(features=self.alpha * x.shape[-1])
        
        # we apply the dense layer to the input
        y = dense(x)

        # the non-linearity is a simple ReLu
        y = nn.relu(y)
                
        # sum the output
        return jnp.sum(y, axis=-1)
    
model = FFN(alpha=1)

vstate = nk.vqs.MCState(sampler, model, n_samples=1008)

In [ ]:
optimizer = nk.optimizer.Sgd(learning_rate=0.1)

# Notice the use, again of Stochastic Reconfiguration, which considerably improves the optimisation
gs = nk.driver.VMC(H, optimizer, variational_state=vstate,preconditioner=nk.optimizer.SR(diag_shift=0.1))

log=nk.logging.RuntimeLog()
gs.run(n_iter=300,out=log)

ffn_energy=vstate.expect(H)
error=abs((ffn_energy.mean-eig_vals[0])/eig_vals[0])
print("Optimized energy and relative error: ",ffn_energy,error)

In [ ]:
data_FFN = log.data

plt.errorbar(data_jastrow["Energy"].iters, data_jastrow["Energy"].Mean, yerr=data_jastrow["Energy"].Sigma, label="Jastrow")
plt.errorbar(data_FFN["Energy"].iters, data_FFN["Energy"].Mean, yerr=data_FFN["Energy"].Sigma, label="FFN")
plt.legend()

plt.xlabel('Iterations')
plt.ylabel('Energy')